In [77]:

from langchain_google_genai import GoogleGenerativeAI as genAI
import os
from dotenv import load_dotenv
load_dotenv()
llm=genAI(model='gemini-1.5-flash',google_api_key=os.environ["GOOGLE_API_KEY"],temperature=0.2)
response=llm.invoke("Write a two line summary on avatar movie")
print(response)

I0000 00:00:1726089199.182735 10008754 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In the lush world of Pandora, a paraplegic marine infiltrates a native tribe to secure a valuable mineral, but finds himself torn between his mission and his growing connection to the alien culture.  His journey leads to a battle for the survival of both Pandora and humanity. 



In [21]:
from langchain.utilities import SQLDatabase

db_user = 'root'
db_password = '########'  # URL-encoded '@' as '%40'
db_host = 'localhost'
db_port = '3306'
db_name = 'V_tshirts'

db_uri = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
db = SQLDatabase.from_uri(db_uri, sample_rows_in_table_info=3)

print(db.table_info)



CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [33]:
from langchain_experimental.sql import SQLDatabaseChain as SQLchain# type: ignore
db_chain= SQLchain.from_llm(llm,db,verbose=True)
q0=db_chain.run('How many nike t-shirts do we have')

/var/folders/3t/d_psg7md26q6mszp_12s84400000gn/T/ipykernel_50639/2273384975.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  q1=db_chain.run('How many nike t-shirts do we have')




> Entering new SQLDatabaseChain chain...
How many nike t-shirts do we have
SQLQuery:Question: How many nike t-shirts do we have
SQLQuery: SELECT COUNT(*) FROM `t_shirts` WHERE `brand` = 'Nike'
SQLResult: [(14,)]
Answer:Question: How many nike t-shirts do we have
SQLQuery:SELECT COUNT(*) FROM `t_shirts` WHERE `brand` = 'Nike'
> Finished chain.


In [43]:
q1=db_chain.run("How many t-shirts do we have left for Nike in XS size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for Nike in XS size and white color?
SQLQuery:Question: How many t-shirts do we have left for Nike in XS size and white color?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White' LIMIT 1
SQLResult: [(12,)]
Answer:Question: How many t-shirts do we have left for Nike in XS size and white color?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White' LIMIT 1
> Finished chain.


In [44]:
q1=db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLQuery:Question: What is the total stock quantity of Nike t-shirts that are white and size XS?
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `color` = 'White' AND `size` = 'XS'
SQLResult: [(Decimal('12'),)]
Answer:Question: What is the total stock quantity of Nike t-shirts that are white and size XS?
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `color` = 'White' AND `size` = 'XS'
> Finished chain.


In [45]:
q2=db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:Question: If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery: SELECT SUM((`t_shirts`.`price` * (
    1 - (`discounts`.`pct_discount` / 100)
))) AS `revenue` FROM `t_shirts` JOIN `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id` WHERE `t_shirts`.`brand` = 'Levi'
SQLResult: [(Decimal('27.900000'),)]
Answer:Question: If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery: SELECT SUM((`t_shirts`.`price` * (1 - (`discounts`.`pct_discount` / 100)))) AS `revenue` FROM `t_shirts` JOIN `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id` WHERE `t_shirts`.`brand` = 'Levi'
> Finished chain.


In [46]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

q2 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:Question: What is the total revenue from Levi t-shirts after applying discounts?
SQLQuery: SELECT SUM(`t_shirts`.`price` * `t_shirts`.`stock_quantity` * (
    1 - (`discounts`.`pct_discount` / 100)
)) AS `total_revenue` FROM `t_shirts` LEFT JOIN `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id` WHERE `t_shirts`.`brand` = 'Levi'
SQLResult: [(Decimal('2566.800000'),)]
Answer:Question: What is the total revenue from Levi t-shirts, taking into account discounts?
SQLQuery: SELECT SUM(`t_shirts`.`price` * `t_shirts`.`stock_quantity` * (1 - (`discounts`.`pct_discount` / 100))) AS `total_revenue` FROM `t_shirts` LEFT JOIN `discounts` ON `t_shirts`.`t_s

In [47]:
q3 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:Question: How many white color Levi's t shirts we have available?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White' LIMIT 1
SQLResult: [(50,)]
Answer:Question: How many white color Levi's t shirts we have available?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White' LIMIT 1
> Finished chain.


In [66]:
q3=db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")
q3



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:Question: What is the total stock quantity of Levi White XL t-shirts?
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White' AND `size` = 'XL'
SQLResult: [(Decimal('92'),)]
Answer:Question: How many white Levi t-shirts are in stock?
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'
> Finished chain.


"Question: How many white Levi t-shirts are in stock?\nSQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'"

### Now the model has made some errors we try to fix that using Few Shot Learning method 

In [49]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : q1},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': q2} ,
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : q3
     }
]

In [52]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [53]:
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query Question: What is the total stock quantity of Nike t-shirts that are white and size XS?\nSQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `color` = 'White' AND `size` = 'XS'",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query Question: What is the total revenue from Levi t-shirts, taking into account discounts?\nSQLQuery: SELECT SUM(`t_shirts`.`price` * `t_shirts`.`stock_quantit

Now I have the embedding and the text so I'll create a vectorDb and store all that in that Db

In [61]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Initialize the HuggingFace embedding model
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Creating the FAISS vector store (FAISS (Facebook AI Similarity Search))
vectorstore = FAISS.from_texts(to_vectorize, embeddings,metadatas=few_shots)


/Users/gnsaivarun/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Pulling similar queries from our Db

In [62]:
from langchain.prompts import SemanticSimilarityExampleSelector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many t-shirts do we have left for Nike in S size and white color?"})

[{'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query',
  'Answer': "Question: What is the total stock quantity of Nike t-shirts that are white and size XS?\nSQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `color` = 'White' AND `size` = 'XS'"},
 {'Question': "How many white color Levi's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query',
  'Answer': "Question: How many white Levi t-shirts are in stock?\nSQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'"}]

Langchain gives sql prompts by default 

In [67]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)


Only use the following tables:
{table_info}

Question: {input}


In [68]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [71]:
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [72]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)


In [74]:
from langchain.prompts import FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [75]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)